# Traveling Wave RF Cavity Test

In [ ]:
import numpy as np
from pmd_beamphysics import single_particle
from pmd_beamphysics.units import mec2

import impact.z as IZ
from impact.z import ImpactZInput

In [ ]:
reference_particle_mass = mec2
energy = 10e6
pz = np.sqrt(energy**2 - mec2**2)

P0 = single_particle(x=1e-3, pz=pz)

In [ ]:
rfdata9 = IZ.load_rfdata_from_file("../../templates/tesla_9cell_cavity/rfdata9")

In [ ]:
input = ImpactZInput(
    initial_particles=P0,
    ncpu_y=1,
    ncpu_z=1,
    seed=-1,
    n_particle=1,
    nx=64,
    ny=64,
    nz=64,
    distribution="read",
    twiss_beta_x=10.0,
    twiss_beta_y=10.0,
    average_current=0.0,
    reference_kinetic_energy=9489001.05,
    reference_particle_mass=reference_particle_mass,
    reference_particle_charge=-1.0,
    reference_frequency=1.0,
    file_data={"TWCAV": rfdata9},
    lattice=[
        IZ.WriteFull(name="initial_particles", map_steps=2000),
        IZ.TravelingWaveRFCavity(
            name="TWCAV",
            length=1.0,
            steps=10,
            map_steps=10,
            field_scaling=2000000.0,
            rf_frequency=1300000000.0,
            radius=1.0,
            file_id=9.0,
        ),
        IZ.WriteFull(name="final_particles", map_steps=2001),
    ],
)

In [ ]:
I = IZ.ImpactZ(input, use_temp_dir=False, workdir="./lcav-tmp", initial_particles=P0)

In [ ]:
output = I.run(verbose=True)

In [ ]:
P = I.output.particles["final_particles"]

In [ ]:
I.output.stats.mean_x

In [ ]:
I.output.stats.mean_y

In [ ]:
I.output["mean_z"]

In [ ]:
output.plot(["mean_x", "mean_y"])

In [ ]:
import matplotlib.pyplot as plt

stats = I.output.stats
z = stats.z
x = stats.mean_x
y = stats.mean_y

fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(8, 5))
ax1.plot(z, x, label="Impact-Z")
ax1.set_ylabel(r"$x$ (m)")

ax2.plot(z, y, label="Impact-Z")
ax2.set_ylabel(r"$y$ (m)")
ax2.set_xlabel(r"$s$ (m)")

ax2.legend()

I.input.plot(ax=ax3);